# Pipeline Implementation

This code will contain the implementation for our pipeline that combines both our object detection and our traffic prediction network

In [ ]:
from object_detection.callable_implementation import write_csv
from object_detection.callable_implementation import process_image
from object_detection.callable_implementation import get_dt_info
from object_detection.callable_implementation import load_model
from object_detection.callable_implementation import load_tensor_img
from object_detection.render_video import images_to_video_ffmpeg

import requests
import cv2
import numpy as np
import imutils
import time
import os
import threading
from queue import Queue
import shutil
import joblib
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [ ]:
# load the model
model, vehicle_classes, device = load_model()
loaded_model = joblib.load("/home/brownjordan317/fall_2024/CSCI443/Github/CSCI-443/traffic_prediction_model.pkl")

In [3]:
# read ip from text file
with open('my_ip.txt', 'r') as f:
    ip = f.read().strip()
print(ip)

172.26.40.147:8080


In [ ]:
def add_counts(data, image, pred, output_folder, image_file):
    # Resize the image
    image = imutils.resize(image, width=800)

    # Get image dimensions
    height, width, _ = image.shape

    # Add white padding to the right side for annotations
    padding_width = 300  # Adjust as needed for the text
    padded_image = np.full((height, width + padding_width, 3), 255, dtype=np.uint8)  # White background
    padded_image[:, :width] = image  # Place the original image on the left

    # Create annotations
    annotations = [
        f"Time: {data['Time']}",
        f"Day: {data['Day of the week']}",
        f"Car Count: {data['CarCount']}",
        f"Bike Count: {data['BikeCount']}",
        f"Bus Count: {data['BusCount']}",
        f"Truck Count: {data['TruckCount']}",
        f"Total: {data['Total']}",
        f"Prediction: {pred}"
    ]

    # Set annotation settings
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.6
    font_color = (0, 0, 0)  # Black text for contrast with white background
    line_type = cv2.LINE_AA

    # Determine the starting position for annotations (right side of the padded image)
    x_offset = width + 10  # Leave a small margin from the start of the padding
    y_offset = 30  # Initial vertical offset
    line_height = 25  # Space between each annotation

    # Add each annotation to the image
    for i, text in enumerate(annotations):
        y_position = y_offset + i * line_height
        cv2.putText(padded_image, text, (x_offset, y_position), font, font_scale, font_color, 1, line_type)

    return image

In [ ]:
RUNNING = True

# Replace the below URL with your own. Make sure to add "/shot.jpg" at the last.
url = f"http://{ip}/shot.jpg"

# Initialize variables
fps = 4  # Desired FPS for the output video
out_dir = "test_images"

# Create the output directory if it doesn't exist
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Queue for storing images that need to be annotated and displayed
image_queue = Queue()
annotated_image_queue = Queue()

# Function to process the image
def annotate(image_path):
    # print(f"Annotating image: {image_path}")
    output_path = 'live_stream'
    image_base_name = os.path.basename(image_path).split('.')[0]
    outfile_name = f'{image_base_name}_annotated.jpg'
    img, img_tensor = load_tensor_img(image_path, device)
    data, out_image = process_image(model, vehicle_classes, img, img_tensor, output_path, outfile_name)
    time, day = get_dt_info()
    data["Time"] = time
    data["Day of the week"] = day
    # send to prediction model
    # print low, normal, high
    write_csv(data, "test.csv")
    
    headers = ['Time', 'Day of the week', 'CarCount', 'BikeCount', 'BusCount', 'TruckCount', 'Total']
    X_pred = [data.get(header, "") for header in headers]
    # convert to df
    X_pred = pd.DataFrame([X_pred], columns=headers)
    le = LabelEncoder()
    X_pred['Day of the week'] = le.fit_transform(X_pred['Day of the week'])

    pred = int(loaded_model.predict(X_pred)[0])
    
    if pred == 0:
        pred = "heavy"
    elif pred == 1:
        pred = "high"
    elif pred == 2:
        pred = "low"
    elif pred == 3:
        pred = "normal"
    
    # Save the annotated image
    annotated_image_path = os.path.join(out_dir, outfile_name)
    out_image = add_counts(data, out_image, pred)
    cv2.imwrite(annotated_image_path, out_image)

    # Put the annotated image path in the annotated queue
    annotated_image_queue.put(annotated_image_path)

# Function to process images from the queue for annotation
def process_annotations():
    while True:
        # Get an image path from the queue
        img_path = image_queue.get()
        if img_path is None:  # Sentinel value to stop the thread
            break
        # Call the annotation function for each image
        annotate(img_path)
        image_queue.task_done()

# Function to display annotated images in a separate thread
def display_annotated_images():
    global RUNNING
    while RUNNING:
        # Get the annotated image path from the queue
        annotated_img_path = annotated_image_queue.get()
        if annotated_img_path is None:  # Sentinel value to stop the thread
            break
        
        # Read and display the annotated image
        annotated_img = cv2.imread(annotated_img_path)
        cv2.imshow("Annotated Image", annotated_img)
        
        # Press Esc key to exit the display window
        if cv2.waitKey(1) == 27:
            RUNNING = False
            break
        annotated_image_queue.task_done()

# Start the annotation processing thread
annotation_thread = threading.Thread(target=process_annotations, daemon=True)
annotation_thread.start()

# Start the display thread
display_thread = threading.Thread(target=display_annotated_images, daemon=True)
display_thread.start()

# Timing control to achieve 30 FPS
prev_time = time.time()

# While loop to continuously fetch data from the URL
count = 0
while RUNNING:
    # Calculate the time difference
    current_time = time.time()

    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img = cv2.imdecode(img_arr, -1)
    img = imutils.resize(img, width=1000, height=1800)

    # Save the image with a timestamp to ensure uniqueness
    filename = f"{out_dir}/image_{count}.jpg"
    cv2.imwrite(filename, img)

    # Add the image path to the queue for annotation
    # if time diff is greater than 1/fps, add to queue
    if (current_time - prev_time) >= 1 / fps:
        image_queue.put(filename)

    # # Display the image on the Android camera window
    # cv2.imshow("Android_cam", img)

    # # Press Esc key to exit
    # if cv2.waitKey(1) == 27:
    #     break

    # Sleep to avoid high CPU usage (sleeping 1ms in case timing isn't perfect)
    time.sleep(1 / fps)
    count += 1
    

# Stop the annotation and display threads gracefully
image_queue.put(None)  # Sentinel to stop the annotation thread
annotation_thread.join()

annotated_image_queue.put(None)  # Sentinel to stop the display thread
display_thread.join()

cv2.destroyWindow("Annotated Image")
cv2.destroyAllWindows()

total_run_time = time.time() - prev_time
num_seconds = int(total_run_time)
# get num images in live_stream folder
num_images = len([name for name in os.listdir("live_stream") if os.path.isfile(os.path.join("live_stream", name))])
estimated_fps = num_images / num_seconds
print(f"Estimated FPS: {estimated_fps}")


KeyboardInterrupt: 

In [ ]:

date_time_info = get_dt_info()
images_to_video_ffmpeg("live_stream", f"live_stream_{date_time_info}.mp4", estimated_fps)

# remove live_stream folder and its contents
shutil.rmtree("live_stream")

2024/12/02 10:02:15.562759 cmd_run.go:1285: WARNING: cannot start document portal: dial unix /run/user/1000/bus: connect: no such file or directory
